<a href="https://colab.research.google.com/github/Marytem/practice_2019/blob/master/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BBC data preprocessing

In [1]:
from subprocess import Popen, PIPE, STDOUT

In [0]:
target = []
text = []

In [0]:
out = Popen(['ls', 'bbc_filetext/'], 
           stdout=PIPE, 
           stderr=STDOUT)
stdout, stderr = out.communicate()
topics = stdout.decode().split('\n')
topics.remove('')
topics

['business', 'entertainment', 'politics', 'sport', 'tech']

In [0]:
for topic in topics:
    out = Popen(['ls', 'bbc_filetext/'+topic+'/'], 
           stdout=PIPE, 
           stderr=STDOUT)
    stdout, stderr = out.communicate()
    files = stdout.decode().split('\n')
    files.remove('')
    print(topic + '___Done___')
    for file in files:
        target.append(topic)
        text.append(open('bbc_filetext/' + topic+ "/" + file, 'r').read().replace('\n', ''))

business___Done___
entertainment___Done___
politics___Done___
sport___Done___
tech___Done___


In [0]:
len(target)

2224

In [0]:
len(text)

2224

In [0]:
open('bbc_target.csv', 'w').write(str(target).strip('[]').replace(', ', '\n'))

23259

In [0]:
import csv
with open('bbc_text.csv', 'w') as f:
    writer = csv.writer(f)
    for article in text:
        writer.writerow([article])

### Webhouse data preprocessing

In [2]:
from subprocess import Popen, PIPE, STDOUT
import pandas as pd 
import numpy as np
import json

In [5]:
out = Popen(['ls', 'webhouse_data/'], 
           stdout=PIPE, 
           stderr=STDOUT)
stdout, stderr = out.communicate()
topics = stdout.decode().split('\n')
topics.remove('')
topics

['entertainment', 'finance', 'politics', 'sports', 'tech', 'travel']

In [6]:
def get_folder_content(folder_path):
    out = Popen(['ls', folder_path], 
           stdout=PIPE, 
           stderr=STDOUT)
    stdout, stderr = out.communicate()
    files = stdout.decode().split('\n')
    files.remove('')
    return files

In [7]:
def process_folder(folder_files, folder_path, text_file_path):   
    
    for file in folder_files:       
        with open(folder_path+'/'+file) as f:
            article = json.load(f)["text"]
        
        open(text_file_path, 'a').write(article.replace('\n', '').replace('"', '').replace(',', '') + "\n")

In [8]:
for topic in topics:
    content = get_folder_content('webhouse_data/'+topic+'/')
    
    print(topic)
    if len(content)<=4:
        for directory in content:
            files = get_folder_content('webhouse_data/'+topic+'/'+directory+'/')
            process_folder(files, 
                           'webhouse_data/'+topic+'/'+directory,
                           'webhouse_processed/text_'+topic+'.csv')
    else:
        process_folder(content, 
                        'webhouse_data/'+topic,
                        'webhouse_processed/text_'+topic+'.csv')
    print('===================done========================')

entertainment
===================done========================
finance
===================done========================
politics
===================done========================
sports
===================done========================
tech
===================done========================
travel
===================done========================


#### Mixing data for final balanced dataset

In [18]:
tech_data = pd.read_csv('webhouse_processed/text_tech.csv', header=None, engine='python', error_bad_lines=False)[0]
tech_data

0        SAN MATEO Calif. The Portland Trail Blazers ar...
1        The ANC has tightened the noose on members who...
2        Nokia and the French Government announce agree...
3        Submitted by admin on 9/21/2015 @ 4:20 PM   Re...
4        DNA analysis identifies long-lost remains of e...
5        BENGALURU: Homegrown e-commerce giant Flipkart...
6        BEIJING: Xiaomi China's leading smartphone mak...
7        3 big developments in new solar technology By ...
8        PARIS Music streaming service Deezer plans to ...
9        - Website builds & sales o Example: Built up a...
10       Windform 3D Printing Material Passes ESA Tests...
11       Fancred announced today the launch of live str...
12       Image Gallery (5 images) Although mobility sco...
13       iiNet’s 2700 call centre staff will be spared ...
14       (Corrects to Tuesday from Wednesday in paragra...
15       * Comment: * Email: (Date:9/23/2015)...  Sept ...
16       Tencent Holdings Ltd. Chief Executive Officer .

In [45]:
target = ['tech' for i in range(tech_data.shape[0])]

In [6]:
sample_size = int(tech_data.shape[0]/(len(topics)-1))
sample_size

8054

In [47]:
# topics.remove('tech')
for topic in topics:
    data = pd.read_csv('webhouse_processed/text_'+topic+'.csv',
                       header=None, engine='python', error_bad_lines=False)
    tech_data = tech_data.append(data.sample(sample_size)[0], ignore_index=True)
    target.extend([topic for i in range(sample_size)])
    print(topic+' added')

entertainment added


Skipping line 18827: field larger than field limit (131072)
Skipping line 23319: field larger than field limit (131072)
Skipping line 26388: field larger than field limit (131072)


finance added
politics added
sports added


Skipping line 2603: field larger than field limit (131072)
Skipping line 6860: field larger than field limit (131072)
Skipping line 17075: field larger than field limit (131072)
Skipping line 19852: field larger than field limit (131072)
Skipping line 20341: field larger than field limit (131072)
Skipping line 21271: field larger than field limit (131072)
Skipping line 21579: field larger than field limit (131072)
Skipping line 30709: field larger than field limit (131072)
Skipping line 34607: field larger than field limit (131072)
Skipping line 34633: field larger than field limit (131072)
Skipping line 36126: field larger than field limit (131072)
Skipping line 38303: field larger than field limit (131072)
Skipping line 41424: field larger than field limit (131072)
Skipping line 42048: field larger than field limit (131072)
Skipping line 44339: field larger than field limit (131072)
Skipping line 44422: field larger than field limit (131072)
Skipping line 45156: field larger than fie

travel added


In [48]:
len(target)

90612

In [49]:
tech_data.shape

(90612,)

In [50]:
open('webhouse_target.csv', 'w').write(str(target).strip('[]').replace(', ', '\n').replace("'", ''))

835643

In [51]:
tech_data.to_csv('webhouse_text.csv', index=False)

/home/maryana/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [17]:
pd.read_csv('webhouse_target.csv', header=None)[0].unique()

array(["'tech'", "'entertainment'", "'finance'", "'politics'", "'sports'",
       "'travel'"], dtype=object)

In [54]:
pd.read_csv('webhouse_text.csv', header=None)

,0
0,SAN MATEO Calif. The Portland Trail Blazers ar...
1,The ANC has tightened the noose on members who...
2,Nokia and the French Government announce agree...
3,Submitted by admin on 9/21/2015 @ 4:20 PM Re...
4,DNA analysis identifies long-lost remains of e...
5,BENGALURU: Homegrown e-commerce giant Flipkart...
6,BEIJING: Xiaomi China's leading smartphone mak...
7,3 big developments in new solar technology By ...
8,PARIS Music streaming service Deezer plans to ...
9,- Website builds & sales o Example: Built up a...
